The almanac (your puzzle input) lists all of the seeds that need to be planted. It also lists what type of soil to use with each kind of seed, what type of fertilizer to use with each kind of soil, what type of water to use with each kind of fertilizer, and so on. Every type of seed, soil, fertilizer and so on is identified with a number, but numbers are reused by each category - that is, soil 123 and fertilizer 123 aren't necessarily related to each other.

The almanac starts by listing which seeds need to be planted: seeds 79, 14, 55, and 13.

The rest of the almanac contains a list of maps which describe how to convert numbers from a source category into numbers in a destination category. That is, the section that starts with seed-to-soil map: describes how to convert a seed number (the source) to a soil number (the destination). This lets the gardener and his team know which soil to use with which seeds, which water to use with which fertilizer, and so on.

Rather than list every source number and its corresponding destination number one by one, the maps describe entire ranges of numbers that can be converted. 
Each line within a map contains three numbers:   

destination range start, source range start, range length.

Consider again the example seed-to-soil map:

50 98 2  
52 50 48  

The first line has a destination range start of 50, a source range start of 98, and a range length of 2. This line means that the source range starts at 98 and contains two values: 98 and 99. The destination range is the same length, but it starts at 50, so its two values are 50 and 51. With this information, you know that seed number 98 corresponds to soil number 50 and that seed number 99 corresponds to soil number 51.

The second line means that the source range starts at 50 and contains 48 values: 50, 51, ..., 96, 97. This corresponds to a destination range starting at 52 and also containing 48 values: 52, 53, ..., 98, 99. So, seed number 53 corresponds to soil number 55.

Any source numbers that aren't mapped correspond to the same destination number. So, seed number 10 corresponds to soil number 10.

So, the entire list of seed numbers and their corresponding soil numbers looks like this:

seed  soil  
0     0  
1     1  
...   ...  
48    48  
49    49  
50    52  
51    53  
...   ...  
96    98  
97    99  
98    50  
99    51  

With this map, you can look up the soil number required for each initial seed number:

Seed number 79 corresponds to soil number 81.  
Seed number 14 corresponds to soil number 14.  
Seed number 55 corresponds to soil number 57.  
Seed number 13 corresponds to soil number 13.  

The gardener and his team want to get started as soon as possible, so they'd like to know the closest location that needs a seed. Using these maps, find the lowest location number that corresponds to any of the initial seeds. To do this, you'll need to convert each seed number through other categories until you can find its corresponding location number. In this example, the corresponding types are:

Seed 79, soil 81, fertilizer 81, water 81, light 74, temperature 78, humidity 78, location 82.  
Seed 14, soil 14, fertilizer 53, water 49, light 42, temperature 42, humidity 43, location 43.  
Seed 55, soil 57, fertilizer 57, water 53, light 46, temperature 82, humidity 82, location 86.  
Seed 13, soil 13, fertilizer 52, water 41, light 34, temperature 34, humidity 35, location 35.  

So, the lowest location number in this example is 35.

What is the lowest location number that corresponds to any of the initial seed numbers?

In [17]:
# destination range start       source range start           range length.

def create_map(filename):
    src_dst_map = {}
    with open(filename) as file:
        for line in file:
            s = line.split()
            span = int(s[2])

            dst_start = int(s[0])
            dst_end = dst_start + span - 1

            src_start = int(s[1])
            src_end = src_start + span - 1

            src_dst_map[(src_start, src_end)] = (dst_start, dst_end)
    return src_dst_map        

def traverse_map(src, dst_map):
    dst = src
    for k, v in dst_map.items():
        if k[0] <= src and src <= k[1]:
            dst = src - k[0] + v[0]
            break 
    return dst

with open('seeds.txt') as file:
    seed_list = file.readline().split()
    
seed_soil_map = create_map('seed-soil.txt')
soil_fert_map = create_map('soil-fertilizer.txt')
fert_water_map = create_map('fertilizer-water.txt')
water_light_map = create_map('water-light.txt')
light_temp_map = create_map('light-temperature.txt')
temp_humid_map = create_map('temperature-humidity.txt')
humid_loc_map = create_map('humidity-location.txt')

results = []
for seed in seed_list:
    soil = traverse_map(int(seed), seed_soil_map)
    fert = traverse_map(soil, soil_fert_map)
    water = traverse_map(fert, fert_water_map)
    light = traverse_map(water, water_light_map)
    temp = traverse_map(light, light_temp_map)
    humid = traverse_map(temp, temp_humid_map)
    loc = traverse_map(humid, humid_loc_map)

    results.append(loc)

print(min(results))

825516882


Everyone will starve if you only plant such a small number of seeds. Re-reading the almanac, it looks like the seeds: line actually describes ranges of seed numbers.
The values on the initial seeds: line come in pairs. Within each pair, the first value is the start of the range and the second value is the length of the range.
Consider all of the initial seed numbers listed in the ranges on the first line of the almanac. What is the lowest location number that corresponds to any of the initial seed numbers?

In [6]:
# destination range start       source range start           range length.

def create_map(filename):
    src_dst_map = {}
    with open(filename) as file:
        for line in file:
            s = line.split()
            span = int(s[2])

            dst_start = int(s[0])
            dst_end = dst_start + span - 1

            src_start = int(s[1])
            src_end = src_start + span - 1

            src_dst_map[(src_start, src_end)] = (dst_start, dst_end)
    return src_dst_map        

def traverse_map(src_list, dst_map):
    dst  = []
    for src in src_list:
        lower = src[0]
        upper = src[1]
        completed = False
        for k, v in dst_map.items():
            if k[0] <= lower and upper <= k[1]:
                dst_low = lower - k[0] + v[0]
                dst_high = upper - k[0] + v[0]
                dst.append((dst_low, dst_high))
                completed = True
                break

            elif lower < k[0] and (k[0] <= upper and upper <= k[1]):
                dst_low = v[0]
                dst_high = upper - k[0] + v[0]
                dst.append((dst_low, dst_high))
                upper = k[0] - 1

            elif (k[0] <= lower and lower <= k[1]) and upper > k[1]:
                dst_low = lower - k[0] + v[0]
                dst_high = v[1]
                dst.append((dst_low, dst_high))
                lower = k[1] + 1
        
        if not completed:
            dst.append((lower, upper))
    return dst     

def print_map(garden_map):
    for k, v in garden_map.items():
        print(k, v)

seeds = []
with open('seeds.txt') as file:
    seed_list = file.readline().split()
    i = 0
    while i < len(seed_list):
        seed = int(seed_list[i])
        span = int(seed_list[i+1])
        seeds.append((seed, seed+span-1))
        i += 2
    
seed_soil_map = create_map('seed-soil.txt')
soil_fert_map = create_map('soil-fertilizer.txt')
fert_water_map = create_map('fertilizer-water.txt')
water_light_map = create_map('water-light.txt')
light_temp_map = create_map('light-temperature.txt')
temp_humid_map = create_map('temperature-humidity.txt')
humid_loc_map = create_map('humidity-location.txt')

soil = traverse_map(seeds, seed_soil_map)
fert = traverse_map(soil, soil_fert_map)
water = traverse_map(fert, fert_water_map)
light = traverse_map(water, water_light_map)
temp = traverse_map(light, light_temp_map)
humid = traverse_map(temp, temp_humid_map)
loc = traverse_map(humid, humid_loc_map)

print(min(loc))

(136096660, 158177717)
